# TPL Data

## Preprocessing GTFS data

**GTFS**: General Transit Feed Specification

The General Transit Feed Specification (GTFS), also known as GTFS static or static transit to differentiate it from the GTFS realtime extension, defines a common format for public transportation schedules and associated geographic information. GTFS "feeds" let public transit agencies publish their transit data and developers write applications that consume that data in an interoperable way.

A GTFS feed is composed of a series of text files collected in a ZIP file. Each file models a particular aspect of transit information: stops, routes, trips, and other schedule data. 
The details of each file are defined in the [GTFS reference](https://developers.google.com/transit/gtfs/reference/)

https://it.wikipedia.org/wiki/General_Transit_Feed_Specification

* **agency**: l'agency corrisponde alla tabella con le informazioni sull'azienda dei trasporti.

agency_name

agency_url

agency_timezone

* **routes**: la tabella routes contiene i percorsi.


route_id (primary key)

route_short_name

route_long_name

route_type --> important: https://developers.google.com/transit/gtfs/reference/extended-route-types

* **trips**:

trip_id (primary key)

route_id (foreign key)

service_id (foreign key)

Campi opzionali:

block_id - Il block ID indica il blocco al quale un viaggio (trip) appartiene.

* **stop_times**: Orari presso una fermata del mezzo di trasporto.

stop_id (primary key)

trip_id (foreign key)

arrival_time

departure_time

stop_sequence

* **stops**: La tabella stops definisce le informazioni geografiche di ogni fermata.

stop_id (primary key)

stop_name

stop_lon

stop_lat

* **calendar**: la tabella calendario definisce la ricorrenza con cui avviene il passaggio di un mezzo di trasporto presso una fermata come i giorni di esercizio ed il periodo di esercizio.

service_id (primary key)

monday

tuesday

wednesday

thursday

friday

saturday

sunday

start_date

end_date


**Tabelle opzionali:**

calendar_dates.txt

fare_attributes.txt

fare_rules.txt

shapes.txt

frequencies.txt

transfers.txt

feed_info.txt

**GTFS DATA:**

* MILANO: https://www.amat-mi.it/it/mobilita/dati-strumenti-tecnologie/dati-gtfs/

* TORINO: http://opendata.5t.torino.it/gtfs/piemonte_it.zip

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import gtfstk as gt
import geopandas as gpd
import sys
from pathlib import Path

In [ ]:
DIR = Path('../..')
sys.path.append(str(DIR))

TPL_DIR = DIR/'data/raw/tpl/'
GEO_DIR = DIR/'data/processed/'

path = GEO_DIR/'Torino_sezioni.geojson'
print (path)
#GEP_PATH =GEO_DIR+'Torino_sezioni.geojson'
#polygon = gpd.read_file(path)

In [ ]:
from references import common_cfg

### Load TPL GTFS Data

In [ ]:
#implementare poi loop tra città
selectedCity = common_cfg.cityList[0]
istatData = common_cfg.get_istat_cpa_data(selectedCity)

In [ ]:
# List feed - hard coded - to improve
if selectedCity=="Torino":
    fileName="torino/piemonte_it.zip"
if selectedCity =="Milano":
    fileName="milano/Export_OpenDataTPL_Current.zip"

path = TPL_DIR/fileName
gt.list_gtfs(path)

In [ ]:
# Read and print feed
feed = gt.read_gtfs(path, dist_units='km')

In [ ]:
# Load stops to intersect with sezioni 
# Multiple routes may use the same stop. The stop_id is used by systems as an internal identifier of this record (e.g., primary key in database), 
# and therefore the stop_id must be dataset unique.
# Keep info "wheelchair_boarding"
stops = feed.stops
#Rename columns to transform in geopandas
stops.rename(columns={"stop_lat": 'latitude', "stop_lon": 'longitude'}, inplace=True)

In [ ]:
#stops_geo: convert csv in geodataframe
stops_geo = common_cfg.df_to_gdf(input_df=stops)

In [ ]:
stops_geo.columns

In [ ]:
istatData.reset_index(level=0, inplace=True)
istatDataMin = istatData[['geometry','IDquartiere','SEZ2011']]

In [ ]:
# We need to filter just on Turin, (we have date for the entire region)
# keep only STOPS in Turin = geometry  + IDquartiere + SEZ2011
# --> spatialjoin using geometry
stops_geo_internal = gpd.sjoin(istatDataMin,stops_geo, how="inner", op='intersects')

In [ ]:
#STOP TIMES gli stops per risalire alla tipologia della linea (metro, bus)
stop_times = feed.stop_times
stop_times = stop_times[["trip_id","stop_id"]]

In [ ]:
#filter just for city stop
stop_times_city = stop_times.loc[stop_times['stop_id'].isin(stops_geo_internal["stop_id"])]

In [ ]:
#mi recupero il trips id per poi andare a prendere il tipo routes
trips = feed.trips
trips = trips[["route_id","trip_id"]]
trips_stop_times_city = trips.merge(stop_times_city, on="trip_id")

In [ ]:
#routee - finale
routes = feed.routes
routes = routes [["route_id","route_type"]]
routes_trips_stop_times_city = routes.merge(trips_stop_times_city, on="route_id" )

In [ ]:
final = routes_trips_stop_times_city[["route_id","route_type","stop_id"]]
final.drop_duplicates(inplace=True)

In [ ]:
final=final.merge(stops_geo_internal, on="stop_id" )

In [ ]:
final = final.drop('geometry', 1)
final = final.drop('stop_timezone',1)
final = final.drop('stop_code', 1)
final = final.drop('index_right',1)

In [ ]:
#Save Final data
outFilename = '../../data/processed/'+selectedCity+'_TPL.csv'
final.to_csv(outFilename, sep=';', index = False)

In [ ]:
final.route_id.nunique()

In [ ]:
final.stop_id.nunique()

In [ ]:
#example with 1871 
#stop_times_TO.loc[stop_times_TO.stop_id=='1871'].trip_id.value_counts()
#test = routes_trips_stop_times_TO.loc[routes_trips_stop_times_TO.stop_id=='1871']
#test.sort_values(['route_id', 'trip_id'], ascending=[True, True])
#test_min = test[["route_id","route_type","stop_id"]]